In [25]:
import argparse
import torch
import os
from scipy.sparse.linalg import inv
from src.utils import *
from src.WLNodeColoring import *
from src.GraphBatching import *
import pickle

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
file_dir = 'data/DBP15K/zh_en/'
lang_num = 2

In [23]:
output_dir = file_dir.replace('data', 'outputs')

In [26]:
os.path.exists(f"{output_dir}/eigen_adj.bin")

False

In [7]:
all_links, all_nodes, adj = load_data(file_dir, lang_num)
adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
norm_adj = adj_normalize(adj + sp.eye(adj.shape[0]))

In [8]:
c = 0.15
eigen_adj = c * inv((sp.eye(adj.shape[0]) - (1 - c) * adj_normalize(adj)).toarray())

In [9]:
wl_node_coloring = WLNodeColoring(all_nodes, all_links)
wl = wl_node_coloring.run()

In [13]:
pickle.dump(eigen_adj, open('outputs/DBP15K/zh_en/eigen_adj.bin', 'wb'), protocol=pickle.HIGHEST_PROTOCOL)

In [22]:
eigen_adj.shape

(38960, 38960)

In [33]:
batching = pickle.load(open('outputs/DBP15K/zh_en/batching/batch_7.bin', 'rb'))

In [51]:
batching_keys = list(batching.keys())

In [38]:
wl = pickle.load(open('outputs/DBP15K/zh_en/wl/wl.bin', 'rb'))

In [43]:
wl_keys = list(wl.keys())

In [53]:
batching[1]

[(1386, 0.06258714025288688),
 (991, 0.05846901245960021),
 (14749, 0.053429957694081605),
 (2569, 0.04856846713365069),
 (12659, 0.03673621199892074),
 (4037, 0.025274942649494755),
 (6137, 0.020548374070412306)]

In [45]:
all_nodes

array([    0,     1,     2, ..., 34457, 34458, 34459], dtype=int32)